### First level analysis codes for BIDS, modified from Russ Poldrack

In [7]:
# Set up
task_name = 'ant1'
task_cond = ['c1-alert','c2-orient','c3-executive']
sess_name = 's02'

fig_plv = 0.05
fig_cor = 'fpr'
fig_in_nb = 'no'

data_dir = '/home/***/Projects/data_CBD/2018_***_DevBrainFuncAtlas'
sub_file = '/home/***/Projects/data_CBD/2018_***_DevBrainFuncAtlas/codes/UnivarActi/age_adult_t1_ant_00.csv'

#### The following sections do not need to be modified

In [8]:
# Import the required library
%matplotlib inline
%config InlineBackend.figure_format = "retina"

import numpy as np
import pandas as pd
import os, json, glob
import nilearn.plotting
import matplotlib.pyplot as mplt
import nipype.algorithms.modelgen as model

from bids import BIDSLayout
from nipype.caching import Memory
from nipype.interfaces import fsl
from nipype.interfaces.base import Bunch
from nistats.thresholding import map_threshold

In [9]:
# Check whether a temporary folder exists
temp_dir = os.path.join(data_dir, 'derivatives', 'temp')
if not os.path.exists(temp_dir): os.makedirs(temp_dir)
mem = Memory(base_dir = temp_dir)

# Check whether results folder exists
lv1res_dir = os.path.join(data_dir, 'derivatives', 'modeling')
if not os.path.exists(lv1res_dir): os.makedirs(lv1res_dir)
    
# Check whether figures folder exists
figure_dir = os.path.join(data_dir, 'derivatives', 'figures')
if not os.path.exists(figure_dir): os.makedirs(figure_dir)

In [10]:
# Acquire image data from BIDS
# layout = BIDSLayout(os.path.join(data_dir, 'datasets'))
# imgfiles = layout.get(suffix = "bold", task = 'em', session = 's02', extension = "nii.gz")

In [11]:
# Repeat for all subjects
sublist = pd.read_csv(sub_file, names = ['id'])

# copes = {}
zmaps = {}
for isub in sublist['id']:
    
    events = pd.read_csv(os.path.join(
        data_dir, "datasets", "sub-%s"%isub, "ses-%s"%sess_name, "func", 
        "sub-%s_ses-%s_task-%s_events.tsv"%(isub, sess_name, task_name)), sep = "\t")
    
    confounds = pd.read_csv(os.path.join(
        data_dir, "derivatives", "fmriprep", "sub-%s"%isub, "ses-%s"%sess_name, "func", 
        "sub-%s_ses-%s_task-%s_desc-confounds_regressors.tsv"%(isub, sess_name, task_name)), 
                            sep = "\t", na_values = "n/a")
    
    tskdsg = [Bunch(
        conditions = ['no', 
                      'double', 
                      'center', 
                      'spatial', 
                      'congruent', 
                      'incongruent'], 
        onsets = [list(events[events.trial_type == 'no'].onset), 
                  list(events[events.trial_type == 'double'].onset), 
                  list(events[events.trial_type == 'center'].onset), 
                  list(events[events.trial_type == 'spatial'].onset), 
                  list(events[events.trial_type == 'congruent'].onset), 
                  list(events[events.trial_type == 'incongruent'].onset)], 
        durations = [list(events[events.trial_type == 'no'].duration), 
                     list(events[events.trial_type == 'double'].duration), 
                     list(events[events.trial_type == 'center'].duration), 
                     list(events[events.trial_type == 'spatial'].duration), 
                     list(events[events.trial_type == 'congruent'].duration), 
                     list(events[events.trial_type == 'incongruent'].duration)], 
        regressors = [list(confounds.csf.fillna(0)), 
                      list(confounds.white_matter.fillna(0))], 
        regressor_names = ['csf', 'white_matter'])]
    
    skip = mem.cache(fsl.ExtractROI)
    skip_res = skip(in_file = os.path.join(
        data_dir, "derivatives", "fmriprep", "sub-%s"%isub, "ses-%s"%sess_name, "func", 
        "sub-%s_ses-%s_task-%s_space-MNI152NLin6Asym_desc-smoothAROMAnonaggr_bold.nii.gz"%(isub, sess_name, task_name)), 
                    t_min = 0, t_size = -1)
    
    json_file = os.path.join(
        data_dir, "datasets", "sub-%s"%isub, "ses-%s"%sess_name, "func", 
        "sub-%s_ses-%s_task-%s_bold.json"%(isub, sess_name, task_name))
    with open(json_file, 'r') as load_file: file_dic = json.load(load_file)
    
    specify = model.SpecifyModel()
    specify.inputs.input_units = 'secs'
    specify.inputs.functional_runs = skip_res.outputs.roi_file
    specify.inputs.time_repetition = file_dic['RepetitionTime']
    specify.inputs.high_pass_filter_cutoff = 128.
    specify.inputs.subject_info = tskdsg
    specify_model_results = specify.run()
    
    cond_alert = ['alert', 'T', ['no', 'double'], [-1, 1]]
    cond_orient = ['orient', 'T', ['center', 'spatial'], [-1, 1]]
    cond_executive = ["executive", 'T', ['congruent', 'incongruent'], [-1, 1]]
    contrasts = [cond_alert, cond_orient, cond_executive]
    
    level1_design = mem.cache(fsl.model.Level1Design)
    level1_design_res = level1_design(
        interscan_interval = file_dic['RepetitionTime'], 
        bases = {'dgamma':{'derivs': True}}, 
        session_info = specify_model_results.outputs.session_info, 
        model_serial_correlations = True, 
        contrasts = contrasts)
    
    modelgen = mem.cache(fsl.model.FEATModel)
    modelgen_res = modelgen(
        fsf_file = level1_design_res.outputs.fsf_files, 
        ev_files = level1_design_res.outputs.ev_files)
    
    mask = mem.cache(fsl.maths.ApplyMask)
    mask_res = mask(in_file = skip_res.outputs.roi_file, 
                    mask_file = os.path.join(
                        data_dir, "derivatives", "fmriprep", "sub-%s"%isub, "ses-%s"%sess_name, "func", 
                        "sub-%s_ses-%s_task-%s_space-MNI152NLin6Asym_desc-brain_mask.nii.gz"%(isub, sess_name, task_name)))
    
    filmgls = mem.cache(fsl.FILMGLS)
    filmgls_res = filmgls(in_file = mask_res.outputs.out_file, 
                          design_file = modelgen_res.outputs.design_file, 
                          tcon_file = modelgen_res.outputs.con_file, 
                          fcon_file = modelgen_res.outputs.fcon_file, 
                          autocorr_noestimate = True)
    
    # copes[isub] = list(filmgls_res.outputs.copes)
    zmaps[isub] = list(filmgls_res.outputs.zstats)

200619-21:58:29,702 nipype.workflow INFO:
	 [Node] Setting-up "bdf25d9a559f020e61ca1e802f09616d" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-utils-ExtractROI/bdf25d9a559f020e61ca1e802f09616d".
200619-21:58:29,704 nipype.workflow INFO:
	 [Node] Cached "bdf25d9a559f020e61ca1e802f09616d" - collecting precomputed outputs
200619-21:58:29,705 nipype.workflow INFO:
	 [Node] "bdf25d9a559f020e61ca1e802f09616d" found cached.
200619-21:58:29,730 nipype.workflow INFO:
	 [Node] Setting-up "c4878297c7e03ad7ee043b8ee0b6f3bc" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-Level1Design/c4878297c7e03ad7ee043b8ee0b6f3bc".
200619-21:58:29,741 nipype.workflow INFO:
	 [Node] Cached "c4878297c7e03ad7ee043b8ee0b6f3bc" - collecting precomputed outputs
200619-21:58:29,742 nipype.workflow INFO:
	 [Node] "c4878297c7e03ad7ee043b8ee0b6f3bc" found cached.
200619-21:58:29,750 nipy

200619-21:58:30,97 nipype.workflow INFO:
	 [Node] Setting-up "5e6081640309318002e7207bd6acacd5" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FEATModel/5e6081640309318002e7207bd6acacd5".
200619-21:58:30,100 nipype.workflow INFO:
	 [Node] Cached "5e6081640309318002e7207bd6acacd5" - collecting precomputed outputs
200619-21:58:30,100 nipype.workflow INFO:
	 [Node] "5e6081640309318002e7207bd6acacd5" found cached.
200619-21:58:30,107 nipype.workflow INFO:
	 [Node] Setting-up "71f49207c8f1cd4151a26647916fde34" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-maths-ApplyMask/71f49207c8f1cd4151a26647916fde34".
200619-21:58:30,109 nipype.workflow INFO:
	 [Node] Cached "71f49207c8f1cd4151a26647916fde34" - collecting precomputed outputs
200619-21:58:30,110 nipype.workflow INFO:
	 [Node] "71f49207c8f1cd4151a26647916fde34" found cached.
200619-21:58:30,119 nipype.wo

200619-21:58:30,489 nipype.workflow INFO:
	 [Node] Setting-up "bcf4bb38009f4cf0a5448f05507c9789" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FILMGLS/bcf4bb38009f4cf0a5448f05507c9789".
200619-21:58:30,495 nipype.workflow INFO:
	 [Node] Cached "bcf4bb38009f4cf0a5448f05507c9789" - collecting precomputed outputs
200619-21:58:30,496 nipype.workflow INFO:
	 [Node] "bcf4bb38009f4cf0a5448f05507c9789" found cached.
200619-21:58:30,529 nipype.workflow INFO:
	 [Node] Setting-up "cf719a18f4bc92446eda865989fc7090" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-utils-ExtractROI/cf719a18f4bc92446eda865989fc7090".
200619-21:58:30,531 nipype.workflow INFO:
	 [Node] Cached "cf719a18f4bc92446eda865989fc7090" - collecting precomputed outputs
200619-21:58:30,532 nipype.workflow INFO:
	 [Node] "cf719a18f4bc92446eda865989fc7090" found cached.
200619-21:58:30,554 nipype.wo

200619-21:58:30,901 nipype.workflow INFO:
	 [Node] Setting-up "2b89bc69c7f7cf0379b6f967b5e00fe6" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-Level1Design/2b89bc69c7f7cf0379b6f967b5e00fe6".
200619-21:58:30,911 nipype.workflow INFO:
	 [Node] Cached "2b89bc69c7f7cf0379b6f967b5e00fe6" - collecting precomputed outputs
200619-21:58:30,912 nipype.workflow INFO:
	 [Node] "2b89bc69c7f7cf0379b6f967b5e00fe6" found cached.
200619-21:58:30,919 nipype.workflow INFO:
	 [Node] Setting-up "e524b6f75b7769c0b944d3b5c7ea1106" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FEATModel/e524b6f75b7769c0b944d3b5c7ea1106".
200619-21:58:30,923 nipype.workflow INFO:
	 [Node] Cached "e524b6f75b7769c0b944d3b5c7ea1106" - collecting precomputed outputs
200619-21:58:30,924 nipype.workflow INFO:
	 [Node] "e524b6f75b7769c0b944d3b5c7ea1106" found cached.
200619-21:58:30,931 nipyp

200619-21:58:31,231 nipype.workflow INFO:
	 [Node] Setting-up "4a43224274056c9ec330b48c1d6d61d0" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-maths-ApplyMask/4a43224274056c9ec330b48c1d6d61d0".
200619-21:58:31,233 nipype.workflow INFO:
	 [Node] Cached "4a43224274056c9ec330b48c1d6d61d0" - collecting precomputed outputs
200619-21:58:31,233 nipype.workflow INFO:
	 [Node] "4a43224274056c9ec330b48c1d6d61d0" found cached.
200619-21:58:31,242 nipype.workflow INFO:
	 [Node] Setting-up "e1fcdcc311c4de677c21d05058aaedd7" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FILMGLS/e1fcdcc311c4de677c21d05058aaedd7".
200619-21:58:31,248 nipype.workflow INFO:
	 [Node] Cached "e1fcdcc311c4de677c21d05058aaedd7" - collecting precomputed outputs
200619-21:58:31,248 nipype.workflow INFO:
	 [Node] "e1fcdcc311c4de677c21d05058aaedd7" found cached.
200619-21:58:31,286 nipype.wor

...

200619-21:58:39,752 nipype.workflow INFO:
	 [Node] Setting-up "016a4aebcd17196802fb133dfb6a366c" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-Level1Design/016a4aebcd17196802fb133dfb6a366c".
200619-21:58:39,761 nipype.workflow INFO:
	 [Node] Cached "016a4aebcd17196802fb133dfb6a366c" - collecting precomputed outputs
200619-21:58:39,761 nipype.workflow INFO:
	 [Node] "016a4aebcd17196802fb133dfb6a366c" found cached.
200619-21:58:39,768 nipype.workflow INFO:
	 [Node] Setting-up "03eb64759eac05dd731b9d0442f77eb0" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FEATModel/03eb64759eac05dd731b9d0442f77eb0".
200619-21:58:39,771 nipype.workflow INFO:
	 [Node] Cached "03eb64759eac05dd731b9d0442f77eb0" - collecting precomputed outputs
200619-21:58:39,771 nipype.workflow INFO:
	 [Node] "03eb64759eac05dd731b9d0442f77eb0" found cached.
200619-21:58:39,777 nipyp

200619-21:58:40,82 nipype.workflow INFO:
	 [Node] Setting-up "235c483c049b751d6c23ee8b69839afd" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-maths-ApplyMask/235c483c049b751d6c23ee8b69839afd".
200619-21:58:40,84 nipype.workflow INFO:
	 [Node] Cached "235c483c049b751d6c23ee8b69839afd" - collecting precomputed outputs
200619-21:58:40,85 nipype.workflow INFO:
	 [Node] "235c483c049b751d6c23ee8b69839afd" found cached.
200619-21:58:40,93 nipype.workflow INFO:
	 [Node] Setting-up "d0bbe75bfe9749d968a99e83933a77a9" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FILMGLS/d0bbe75bfe9749d968a99e83933a77a9".
200619-21:58:40,99 nipype.workflow INFO:
	 [Node] Cached "d0bbe75bfe9749d968a99e83933a77a9" - collecting precomputed outputs
200619-21:58:40,99 nipype.workflow INFO:
	 [Node] "d0bbe75bfe9749d968a99e83933a77a9" found cached.
200619-21:58:40,135 nipype.workflow 

200619-21:58:40,432 nipype.workflow INFO:
	 [Node] Setting-up "7333b312c00d5f485caca55c91a44194" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-utils-ExtractROI/7333b312c00d5f485caca55c91a44194".
200619-21:58:40,434 nipype.workflow INFO:
	 [Node] Cached "7333b312c00d5f485caca55c91a44194" - collecting precomputed outputs
200619-21:58:40,435 nipype.workflow INFO:
	 [Node] "7333b312c00d5f485caca55c91a44194" found cached.
200619-21:58:40,456 nipype.workflow INFO:
	 [Node] Setting-up "c8a025812030727c00115f39d2773793" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-Level1Design/c8a025812030727c00115f39d2773793".
200619-21:58:40,465 nipype.workflow INFO:
	 [Node] Cached "c8a025812030727c00115f39d2773793" - collecting precomputed outputs
200619-21:58:40,466 nipype.workflow INFO:
	 [Node] "c8a025812030727c00115f39d2773793" found cached.
200619-21:58:40,472 nipy

200619-21:58:40,779 nipype.workflow INFO:
	 [Node] Setting-up "830e1a6c65740f0a28617b94b1feb795" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FEATModel/830e1a6c65740f0a28617b94b1feb795".
200619-21:58:40,781 nipype.workflow INFO:
	 [Node] Cached "830e1a6c65740f0a28617b94b1feb795" - collecting precomputed outputs
200619-21:58:40,782 nipype.workflow INFO:
	 [Node] "830e1a6c65740f0a28617b94b1feb795" found cached.
200619-21:58:40,787 nipype.workflow INFO:
	 [Node] Setting-up "0b5f57c8e8b3071f8e4b39f0f3c3806b" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-maths-ApplyMask/0b5f57c8e8b3071f8e4b39f0f3c3806b".
200619-21:58:40,789 nipype.workflow INFO:
	 [Node] Cached "0b5f57c8e8b3071f8e4b39f0f3c3806b" - collecting precomputed outputs
200619-21:58:40,789 nipype.workflow INFO:
	 [Node] "0b5f57c8e8b3071f8e4b39f0f3c3806b" found cached.
200619-21:58:40,798 nipype.w

200619-21:58:41,101 nipype.workflow INFO:
	 [Node] Setting-up "7ff80fd827ff266ddaa2301f59c4f042" in "/home/haolei/Projects/data_CBD/2018_Hao_DevBrainFuncAtlas/derivatives/temp/nipype_mem/nipype-interfaces-fsl-model-FILMGLS/7ff80fd827ff266ddaa2301f59c4f042".
200619-21:58:41,106 nipype.workflow INFO:
	 [Node] Cached "7ff80fd827ff266ddaa2301f59c4f042" - collecting precomputed outputs
200619-21:58:41,107 nipype.workflow INFO:
	 [Node] "7ff80fd827ff266ddaa2301f59c4f042" found cached.


In [12]:
for icond in range(len(task_cond)):
    for isub, iimg in zmaps.items(): 
        smooth_zmap = nilearn.image.smooth_img(iimg[icond], 6)
        if fig_plv == 1: threshold = None
        else: _, threshold = map_threshold(smooth_zmap, alpha = fig_plv, height_control = fig_cor)
        
        if fig_in_nb == 'no': 
            nilearn.plotting.plot_glass_brain(
                smooth_zmap, display_mode = 'ortho', colorbar = True, threshold = threshold, plot_abs = False, 
                title = '%s_%s_%s_%s_%s'%(task_name, task_cond[icond], fig_cor, str(fig_plv), isub), 
                output_file = os.path.join(figure_dir, '%s_%s_%s_%s_%s.png'%(task_name, task_cond[icond], fig_cor, str(fig_plv), isub)))
        elif fig_in_nb == 'yes': 
            nilearn.plotting.plot_glass_brain(
                smooth_zmap, display_mode = 'ortho', colorbar = True, threshold = threshold, plot_abs = False, 
                title = '%s_%s_%s_%s_%s'%(task_name, task_cond[icond], fig_cor, str(fig_plv), isub), 
                axes = [1, 1, 0.8, 0.8])